In [2]:
import pandas as pd
import tushare as ts
import requests
import json
import datetime

In [12]:
def get_qfq(code: str, target_date, start_date=None, end_date='30221011') -> float:
    try:
        if target_date.weekday() in (5,6):
            print(f"qfq: {target_date} 是周末，往后推到周一")
            target_date = target_date + datetime.timedelta(7 - target_date.weekday())
        try:
            target_date = target_date.strftime("%Y%m%d")
        except Exception as e:
            print(f"code: {code}, date: {target_date} 时间戳错误，返回0")
            return 0
        target_date = target_date.replace('-', '')
        start_date = str(start_date) if start_date else str(target_date)
        end_date = str(end_date)

        df = ts.pro_bar(ts_code=code, adj='qfq', start_date=start_date, end_date=end_date)
        try:
            res = df.loc[df['trade_date'] == target_date, ['ts_code', 'trade_date', 'close']]
            return res.close.values[0]
        except Exception as e:
            print(f"code: {code}, date: {target_date}, 没数据，返回0")
            return 0
    except Exception as e:
        print(f"code: {code}, date: {target_date}, 未知错误，返回0, error: {e}")
        return 0

In [13]:
def get_hs300(target_date, start_date=None, end_date='30221011') -> float:
    try:
        if target_date.weekday() in (5,6):
            print(f"hs300: {target_date} 是周末，往后推到周一")
            target_date = target_date + datetime.timedelta(7 - target_date.weekday())
        try:
            target_date = target_date.strftime("%Y-%m-%d")
        except Exception as e:
            print(f"code: {code}, date: {target_date} 时间戳错误，返回0")
            return 0
        hs300 = ts.get_hist_data('hs300', start='20190101', end=end_date)
        try:
            return hs300.loc[target_date].close
        except Exception as e:
            print(f"{target_date} 没有沪深的数据，返回0. 错误码：{e}")
            return 0
    except Exception as e:
        print(f"date: {target_date}, 未知错误，返回0, error: {e}")
        return 0

In [14]:
getAccessTokenUrl = 'https://ft.10jqka.com.cn/api/v1/get_access_token' 
refreshToken = 'eyJzaWduX3RpbWUiOiIyMDIyLTA5LTI5IDEzOjQzOjQ2In0=.eyJ1aWQiOiI1ODUyMjc4ODEifQ==.5CC83A67006931E5D2C8A42DBA046B64170CB9B1D6E0EC198ECAC16065718AA3' 
getAccessTokenHeader = {"Content-Type":"application/json","refresh_token":refreshToken}
getAccessTokenResponse = requests.post(url=getAccessTokenUrl,headers=getAccessTokenHeader) 
accessToken = json.loads(getAccessTokenResponse.content)['data']['access_token'] 

def get_from_ths(code):
    try:
        thsUrl = 'https://ft.10jqka.com.cn/ds_service/api/v1/real_time_quotation'
        thsHeaders = {"Content-Type":"application/json","access_token":accessToken}
        target_indicator = "totalShares,totalCapital,pe_ttm,pb"
        thsPara = {"codes":code,"indicators":target_indicator}
        thsResponse = requests.post(url=thsUrl,json=thsPara,headers=thsHeaders)
        try:
            rsp = json.loads(thsResponse.text)['tables'][0]['table']
            return rsp['totalShares'][0],rsp['totalCapital'][0],rsp['pe_ttm'][0],rsp['pb'][0]
        except Exception as e:
            print(f"code {code} 拿不到同花顺数据。返回 [0,0,0,0], 报错码: {e}, 返回: {thsResponse.text}")
            return 0,0,0,0
    except Exception as e:
        print(f"date: {target_date}, 未知错误，返回[0,0,0,0], error: {e}")
        return 0,0,0,0

In [17]:
target_excel_name = './data/dxzf0930.xlsx'
writer = pd.ExcelWriter(target_excel_name)
for i in (0,1,2):
    print(f'process excel {i}')
    raw_data= pd.read_excel(target_excel_name,header=0,sheet_name=i)
    target_cols = ['issue_price','lift_price','lift6m_price','issue_hs','lift_hs','lift6m_hs','totalShares','totalCapital','pe_ttm','pb']
    for col in target_cols:
        raw_data[col] = 0
    for row in raw_data.iterrows():
        info = row[1]
        this_index = row[0]
        code = info['代码']
        print(f"-------------index: {this_index}, code: {code}-------------")
        issue_date = info['定向增发发行日']
        ban_date = info['解禁日']
        ban6m_date = info['解禁日6个月后']
        index = row[0]
        issue_price = get_qfq(code, issue_date)
        issue_hs300 = get_hs300(issue_date)
        ban_price = get_qfq(code, ban_date)
        ban_hs300 = get_hs300(ban_date)
        ban6m_price = get_qfq(code, ban6m_date)
        ban6m_hs300 = get_hs300(ban6m_date)
        totalShares, totalCapital,pe_ttm,pb = get_from_ths(code)
        raw_data.loc[index,'issue_price'] = issue_price
        raw_data.loc[index,'lift_price'] = ban_price
        raw_data.loc[index,'lift6m_price'] = ban6m_price
        raw_data.loc[index,'issue_hs'] = issue_hs300
        raw_data.loc[index,'lift_hs'] = ban_hs300
        raw_data.loc[index,'lift6m_hs'] = ban6m_hs300
        raw_data.loc[index,'totalShares'] = totalShares
        raw_data.loc[index,'totalCapital'] = totalCapital
        raw_data.loc[index,'pe_ttm'] = pe_ttm
        raw_data.loc[index,'pb'] = pb
    raw_data.to_excel(writer, sheet_name=f"sheet_{i}",index=False)
writer.save()

process excel 0
-------------index: 0, code: 000066.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
code: 000066.SZ, date: 20230215, 没数据，返回0
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2023-02-15 没有沪深的数据，返回0. 错误码：'2023-02-15'
-------------index: 1, code: 000877.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
qfq: 2023-02-25 00:00:00 是周末，往后推到周一
code: 000877.SZ, date: 20230227, 没数据，返回0
hs300: 2023-02-25 00:00:00 是周末，往后推到周一
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2023-02-27 没有沪深的数据，返回0. 错误码：'2023-02-27'
-------------index: 2, code: 300674.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
qfq: 2023-02-12 00:00:00 是周末，往后推到周一
code: 300674.SZ, date: 20230213, 没数据，返回0
hs300: 2023-02-12 00:00:00 是周末，往后推到周一
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
qfq: 2023-03-26 00:00:00 是周末，往后推到周一
code: 002921.SZ, date: 20230327, 没数据，返回0
hs300: 2023-03-26 00:00:00 是周末，往后推到周一
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2023-03-27 没有沪深的数据，返回0. 错误码：'2023-03-27'
-------------index: 26, code: 603518.SH-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
qfq: 2023-03-19 00:00:00 是周末，往后推到周一
code: 603518.SH, date: 20230320, 没数据，返回0
hs300: 2023-03-19 00:00:00 是周末，往后推到周一
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2023-03-20 没有沪深的数据，返回0. 错误码：'2023-03-20'
-------------index: 27, code: 300254.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
code: 300254.SZ, date: 20221110, 没数据，返回0
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2022-11-10 没有沪深的数据，返回0. 错误码：'2022-11-10'
code: 300254.SZ, date: 20230510, 没数据，返回0
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2023-05-10 没有沪深的数据，返回0. 错误码：'2023-0

2023-05-01 没有沪深的数据，返回0. 错误码：'2023-05-01'
-------------index: 49, code: 603956.SH-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
code: 603956.SH, date: 20221025, 没数据，返回0
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2022-10-25 没有沪深的数据，返回0. 错误码：'2022-10-25'
code: 603956.SH, date: 20230425, 没数据，返回0
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2023-04-25 没有沪深的数据，返回0. 错误码：'2023-04-25'
-------------index: 50, code: 688188.SH-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
code: 688188.SH, date: 20221019, 没数据，返回0
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2022-10-19 没有沪深的数据，返回0. 错误码：'2022-10-19'
code: 688188.SH, date: 20230419, 没数据，返回0
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2023-04-19 没有沪深的数据，返回0. 错误码：'2023-04-19'
-------------index: 51, code: 603041.SH-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
code: 603041.SH, date: 20221028, 没数据，返回0
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2022-10

code: 688305.SH, date: 20230731, 没数据，返回0
hs300: 2023-07-30 00:00:00 是周末，往后推到周一
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2023-07-31 没有沪深的数据，返回0. 错误码：'2023-07-31'
-------------index: 71, code: 001896.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
code: 001896.SZ, date: 20230106, 没数据，返回0
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2023-01-06 没有沪深的数据，返回0. 错误码：'2023-01-06'
code: 001896.SZ, date: 20230706, 没数据，返回0
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2023-07-06 没有沪深的数据，返回0. 错误码：'2023-07-06'
-------------index: 72, code: 000798.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
code: 000798.SZ, date: 20230106, 没数据，返回0
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2023-01-06 没有沪深的数据，返回0. 错误码：'2023-01-06'
code: 000798.SZ, date: 20230706, 没数据，返回0
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2023-07-06 没有沪深的数据，返回0. 错误码：'2023-07-06'
-------------index: 73, code: 300750.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：

2023-01-30 没有沪深的数据，返回0. 错误码：'2023-01-30'
qfq: 2023-07-30 00:00:00 是周末，往后推到周一
code: 603187.SH, date: 20230731, 没数据，返回0
hs300: 2023-07-30 00:00:00 是周末，往后推到周一
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2023-07-31 没有沪深的数据，返回0. 错误码：'2023-07-31'
-------------index: 91, code: 300473.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
code: 300473.SZ, date: 20230220, 没数据，返回0
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2023-02-20 没有沪深的数据，返回0. 错误码：'2023-02-20'
qfq: 2023-08-20 00:00:00 是周末，往后推到周一
code: 300473.SZ, date: 20230821, 没数据，返回0
hs300: 2023-08-20 00:00:00 是周末，往后推到周一
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2023-08-21 没有沪深的数据，返回0. 错误码：'2023-08-21'
-------------index: 92, code: 002841.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
code: 002841.SZ, date: 20230213, 没数据，返回0
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2023-02-13 没有沪深的数据，返回0. 错误码：'2023-02-13'
qfq: 2023-08-13 00:00:00 是周末，往后推到周一
code: 002841.SZ, date: 

...

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 104, code: 603232.SH-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 105, code: 002822.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 106, code: 601016.SH-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 107, code: 300223.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
qfq: 2021-09-11 00:00:00 是周末，往后推到周一
hs300: 2021-09-11 00:00:00 是周末，往后推到周一
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 1

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
qfq: 2021-11-13 00:00:00 是周末，往后推到周一
hs300: 2021-11-13 00:00:00 是周末，往后推到周一
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 138, code: 002845.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
qfq: 2021-11-06 00:00:00 是周末，往后推到周一
hs300: 2021-11-06 00:00:00 是周末，往后推到周一
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 139, code: 000625.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 140, code: 603019.SH-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
qfq: 2021-11-06 00:00:00 是周末，往后推到周一
hs300: 2021-11-06 00:00:00 是周末，往后推到周一
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 141, code: 60

hs300: 2021-11-27 00:00:00 是周末，往后推到周一
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 171, code: 603218.SH-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
qfq: 2021-11-27 00:00:00 是周末，往后推到周一
hs300: 2021-11-27 00:00:00 是周末，往后推到周一
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 172, code: 603990.SH-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
qfq: 2021-12-04 00:00:00 是周末，往后推到周一
hs300: 2021-12-04 00:00:00 是周末，往后推到周一
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 173, code: 603078.SH-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 174, code: 002863.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
qfq: 2022-01-09 00:00:00 是周末，往后推到周一
hs300: 2022-01-09 00:00:00 是周末，往后推到周一
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 205, code: 688001.SH-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 206, code: 000723.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 207, code: 603005.SH-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 208, code: 601066.SH-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口

-------------index: 238, code: 002853.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 239, code: 300721.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 240, code: 002149.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
code: 002149.SZ, date: 20220203, 没数据，返回0
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
2022-02-03 没有沪深的数据，返回0. 错误码：'2022-02-03'
-------------index: 241, code: 300648.SZ-------------
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
-------------index: 242, code: 002054.SZ-------------
本接口即将停止更新，请尽